In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import accuracy_score
from scipy.special import softmax as softmax
import matplotlib.pyplot as plt

from dataloader.builder import build_dataset
from experiment_setup import build_estimator
from uncertainty_estimator.masks import build_masks, DEFAULT_MASKS
from analysis.metrics import uq_ndcg, ndcg, dcg

from model.cnn import SimpleConv, Trainer

In [ ]:
config = {
    'use_cuda': True,
    'batch_size': 128,
    'epochs': 3,
    'lr': 1e-2,
    'momentum': 0.5,
    'seed': 1,
    'log_interval': 10,
    'nn_runs': 11,
    'dropout_uq': 0.5
}



#### Load data and preprocess

In [ ]:
mnist = build_dataset('mnist', val_size=10_000)
x_train, y_train = mnist.dataset('train')
x_val, y_val = mnist.dataset('val')

ood = build_dataset('fashion_mnist', val_size=0)
x_ood, _ = ood.dataset('train') 

In [ ]:
x_train = x_train.reshape(-1, 1, 28, 28)
x_val = x_val.reshape(-1, 1, 28, 28)
x_ood = x_ood.reshape(-1, 1, 28, 28)
y_train = y_train.astype('long').reshape(-1)
y_val = y_val.astype('long').reshape(-1)
x_train /= 255.0
x_val /= 255.0
x_ood /= 255.0


#### Train model

In [ ]:
train_samples = 500
unique, counts = np.unique(y_train[:train_samples], return_counts=True)
dict(zip(unique, counts))

In [ ]:
model = SimpleConv()
trainer = Trainer(model)
trainer.fit(x_train[:train_samples], y_train[:train_samples], epochs=config['epochs'])
accuracy_score(y_val, trainer.predict(x_val))


### BALD

##### UQ by different masks

In [ ]:
masks = build_masks(DEFAULT_MASKS)


In [ ]:
estimation_samples = train_samples 
uqs, datasets, mask_type = [], [], []

for mask_name, mask in masks.items():
    estimator = build_estimator(
        'bald_masked', trainer, nn_runs=config['nn_runs'], dropout_mask=mask,
        dropout_rate=config['dropout_uq'], num_classes=10)

    for data_name, x_current in (('train', x_train), ('val', x_val), ('ood', x_ood)):
        uq = estimator.estimate(x_current[:estimation_samples])
        uqs = np.concatenate((uqs, uq))
        datasets = np.concatenate((datasets, [data_name]*estimation_samples))
        mask_type = np.concatenate((mask_type, [mask_name]*estimation_samples))


In [ ]:
plt.figure(figsize=(16, 9))

df = pd.DataFrame({'uq': uqs, 'dataset': datasets, 'mask_type': mask_type})
sns.boxplot(data=df, x='dataset',  y='uq', hue='mask_type')



In [ ]:
pool_size = 300
x_pool, y_pool = x_val[:pool_size], y_val[:pool_size]

def ll(trainer, x, y):
    trainer.eval()
    logits = trainer(x).detach().cpu().numpy()
    probs = softmax(logits, axis=-1)[np.arange(len(x)), y]
    return np.log(probs)

pool_ll = ll(trainer, x_pool, y_pool)
    

In [ ]:
plt.figure(figsize=(16, 18))
for name, mask in masks.items():
    estimator = build_estimator(
        'bald_masked', trainer, nn_runs=config['nn_runs'], dropout_mask=mask,
        dropout_rate=config['dropout_uq'], num_classes=10)
    estimations = estimator.estimate(x_pool)
    plt.scatter(estimations, pool_ll, label=name, alpha=0.5)
    plt.xlabel('Uncertainty estimation')
    plt.ylabel('Log likelihood')
plt.legend(loc='lower right')
    

In [ ]:
masks = build_masks(nn_runs=100)
plt.figure(figsize=(10, 25))
num = (len(masks) + 1) // 2
for i, (name, mask) in enumerate(masks.items()):
    plt.subplot(num, 2, i+1)
    estimator = build_estimator(
        'bald_masked', trainer, nn_runs=config['nn_runs'], dropout_mask=mask,
        dropout_rate=config['dropout_uq'], num_classes=10)
    estimations = estimator.estimate(x_pool)
    plt.scatter(estimations, pool_ll, alpha=0.5)
    plt.xlabel('Uncertainty estimation')
    plt.ylabel('Log likelihood')
    plt.title(name)
# plt.legend(loc='lower right')
    
 

In [ ]:
masks = build_masks(nn_runs=100)
estimation_samples = 3000 
ndcgs, mask_type = [], []

for i in range(10):
    print(i)
    idxs = np.random.choice(len(x_train), estimation_samples, replace=False)
    x_current = x_train[idxs]
    y_current = y_train[idxs]

    current_ll = ll(trainer, x_current, y_current)
    for mask_name, mask in masks.items():
        estimator = build_estimator(
            'bald_masked', trainer, nn_runs=config['nn_runs'], dropout_mask=mask,
            dropout_rate=config['dropout_uq'], num_classes=10)
        uq = estimator.estimate(x_current)
        ndcgs.append(uq_ndcg(-current_ll, uq))
        mask_type.append(mask_name)
        estimator.reset()
    
    

In [ ]:
plt.figure(figsize=(16, 9))

df = pd.DataFrame({'ndcg': ndcgs, 'mask_type': mask_type})
sns.boxplot(data=df, x='mask_type',  y='ndcg')

